### Imports

In [1]:
import glob
import numpy as np
import xarray as xr
import pandas as pd
import metpy.calc

In [2]:
from dask_jobqueue import PBSCluster
from distributed import Client
from dask import delayed

### File paths

In [3]:
era5_path = '/glade/campaign/collections/rda/data/d633000/e5.oper.an.pl'
prof449_path = "/glade/campaign/collections/gdex/data/special_projects/pythia_2025/eol-cookbook/m2hats_iss2_data/prof449Mhz_30min_winds"
lustre_scratch  = "/lustre/desc1/scratch/myasears"

### Spin up a cluster

In [ ]:
cluster = PBSCluster(
        job_name = 'dask-eol-25',
        cores = 1,
        memory = '4GiB',
        processes = 1,
        local_directory = lustre_scratch + '/dask/spill',
        log_directory = lustre_scratch + '/dask/logs/',
        resource_spec = 'select=1:ncpus=1:mem=4GB',
        queue = 'casper',
        walltime = '3:00:00',
        interface = 'ext')

In [5]:
client = Client(cluster)

In [6]:
# Scale the cluster and display cluster dashboard URL
n_workers = 5
cluster.scale(n_workers)
client.wait_for_workers(n_workers = n_workers)
cluster

PBSCluster(9d3a0c79, 'tcp://128.117.208.94:34311', workers=5, threads=5, memory=20.00 GiB)

### Load ERA5 data

In [10]:
target_lat = 38.0
target_lon = 243.0

start_date = pd.Timestamp("2023-07-11T00:00:00")
end_date = pd.Timestamp("2023-09-27T23:59:59")
yyyymm = ["202307", "202308", "202309"]

var_map = {"Z": "e5.oper.an.pl.128_129_z",
           "U": "e5.oper.an.pl.128_131_u",
           "V": "e5.oper.an.pl.128_132_v",
           "W": "e5.oper.an.pl.128_135_w"
           }

In [11]:
def open_variable(varname, file_prefix, yyyymm):
    files = []
    for month in yyyymm:
        files.extend(sorted(glob.glob(f'{era5_path}/{month}/{file_prefix}*')))

    ds = xr.open_mfdataset(files, combine="by_coords", parallel=True)

    # Subset to single lat/lon point and campaign time slice
    ds_point = ds[varname].sel(latitude=target_lat, longitude=target_lon, time=slice(start_date, end_date))

    return ds_point

In [12]:
points = []
for varname, file_prefix in var_map.items():
    points.append(open_variable(varname, file_prefix, yyyymm))

# Merge them together
combined_era5 = xr.merge(points)
combined_era5

<xarray.Dataset> Size: 1MB
Dimensions:    (level: 37, time: 1896)
Coordinates:
    latitude   float64 8B 38.0
  * level      (level) float64 296B 1.0 2.0 3.0 5.0 ... 925.0 950.0 975.0 1e+03
    longitude  float64 8B 243.0
  * time       (time) datetime64[ns] 15kB 2023-07-11 ... 2023-09-27T23:00:00
Data variables:
    Z          (time, level) float32 281kB dask.array<chunksize=(1, 37), meta=np.ndarray>
    U          (time, level) float32 281kB dask.array<chunksize=(1, 37), meta=np.ndarray>
    V          (time, level) float32 281kB dask.array<chunksize=(1, 37), meta=np.ndarray>
    W          (time, level) float32 281kB dask.array<chunksize=(1, 37), meta=np.ndarray>
Attributes: (12/14)
    long_name:                                          Geopotential
    short_name:                                         z
    units:                                              m**2 s**-2
    original_format:                                    WMO GRIB 1 with ECMWF...
    ecmwf_local_table:                                  128
    ecmwf_parameter:                                    129
    ...                                                 ...
    grid_specification:                                 0.25 degree x 0.25 de...
    rda_dataset:                                        ds633.0
    rda_dataset_url:                                    https:/rda.ucar.edu/d...
    rda_dataset_doi:                                    DOI: 10.5065/BH6N-5N20
    rda_dataset_group:                                  ERA5 atmospheric pres...
    QuantizeGranularBitGroomNumberOfSignificantDigits:  7

### Load 449 data

In [13]:
files = []
files.extend(sorted(glob.glob(f'{prof449_path}/*.nc')))

In [14]:
# Retrieve common height grid (with a step of 100m) using max and min values
step = 100
min_height = min(xr.open_dataset(f).height.min().item() for f in files)
max_height = max(xr.open_dataset(f).height.max().item() for f in files)

# Create common height grid
common_agl = np.arange(min_height, max_height + step, step)

# Retrieve altitude value from fifth file (after setup -- checked manually)
altitude = xr.open_dataset(files[5]).alt.values

# Use alt to create common MSL, geopotential grids
common_msl = common_agl + altitude
common_geopotential = (9.80665 * 6371008.7714 * common_msl) / (6371008.7714 + common_msl)

In [15]:
def open_and_regrid(f, common_agl, common_msl, common_geopotential):
    ds = xr.open_dataset(f, chunks={})
    
    msl_height = ds['height'].isel(time=0) + altitude
    geopotential = (9.80665 * 6371008.7714 * msl_height) / (6371008.7714 + msl_height)
    
    height_1d = ds['height'].isel(time=0).values
    ds = ds.assign_coords(height=("height", height_1d))
    ds = ds.reindex(height=common_agl)
    
    # Update coords to the reindexed grid
    ds = ds.assign_coords(
        height_agl=("height", common_agl),
        height_msl=("height", common_msl),
        geopotential=("height", common_geopotential)
    )
    
    # Metadata
    ds.height_agl.attrs.update({"long_name": "Height above ground level", "units": "meters"})
    ds.height_msl.attrs.update({"long_name": "Height above mean sea level", "units": "meters"})
    ds.geopotential.attrs.update({"long_name": "Geopotential", "units": "m**2 s**-2"})
    
    # Swap to make geopotential the vertical coordinate
    ds = ds.swap_dims({"height": "geopotential"}).drop_vars("height")

    return ds

In [16]:
datasets = [delayed(open_and_regrid)(f, common_agl, common_msl, common_geopotential) for f in files[2:]]
datasets = [d.compute() for d in datasets]
combined_profiler = xr.concat(datasets, dim="time")
combined_profiler

<xarray.Dataset> Size: 16MB
Dimensions:       (time: 3696, geopotential: 97)
Coordinates:
  * time          (time) datetime64[ns] 30kB 2023-07-13T00:15:00 ... 2023-09-...
    height_agl    (geopotential) float64 776B 200.0 300.0 ... 9.7e+03 9.8e+03
    height_msl    (geopotential) float64 776B 1.841e+03 1.941e+03 ... 1.144e+04
  * geopotential  (geopotential) float64 776B 1.805e+04 1.903e+04 ... 1.12e+05
Data variables: (12/18)
    base_time     (time) datetime64[ns] 30kB 2023-07-13 ... 2023-09-27
    lat           (time) float32 15kB 38.04 38.04 38.04 ... 38.04 38.04 38.04
    lon           (time) float32 15kB -117.1 -117.1 -117.1 ... -117.1 -117.1
    alt           (time) float32 15kB 1.641e+03 1.641e+03 ... 1.641e+03
    frequency     (time) float32 15kB 448.6 448.6 448.6 ... 449.0 449.0 449.0
    x_spacing     (time) float32 15kB 100.0 100.0 100.0 ... 100.0 100.0 100.0
    ...            ...
    snrw          (time, geopotential) float32 1MB dask.array<chunksize=(48, 46), meta=np.ndarray>
    specWid       (time, geopotential) float32 1MB dask.array<chunksize=(48, 46), meta=np.ndarray>
    u_dispersion  (time, geopotential) float32 1MB dask.array<chunksize=(48, 46), meta=np.ndarray>
    v_dispersion  (time, geopotential) float32 1MB dask.array<chunksize=(48, 46), meta=np.ndarray>
    w_dispersion  (time, geopotential) float32 1MB dask.array<chunksize=(48, 46), meta=np.ndarray>
    cons_npoints  (time, geopotential) float32 1MB dask.array<chunksize=(48, 46), meta=np.ndarray>
Attributes: (12/16)
    description:        Modular Profiler consensus winds
    Processing-Method:  30 minute consensus averages
    Processing-Ops:       read_raw_winds mapr_nc szl yyyy correct_szl correct...
    history:            write_cons_5.pro
    author:             Bill Brown, NCAR/EOL
    date:               Mon Jul  1 09:54:45 2024   1719849285
    ...                 ...
    project:            M2HATS
    site:               Tonopah Airport, NV
    missing_value:      -999.0
    zebra_platform:     MAPR/449 winds
    first_file:         /scr/isf_apg/raw/m2hats/iss1/modular_prof/reprocessed...
    last_file:          /scr/isf_apg/raw/m2hats/iss1/modular_prof/reprocessed...

### Combine the datasets

ERA5 data measures on constant pressure; profiler data is measured on constant height. Will need to consider this factor when combining datasets, likely interpolating ERA5 to a common height and assigning this as a dependent coordinate before combination. Thinking of other combination methods in the meantime. 

https://unidata.github.io/MetPy/latest/api/generated/metpy.calc.geopotential_to_height.html